In [1]:
import pickle

In [2]:
from sklearn.preprocessing import FunctionTransformer

In [3]:
import requests, zipfile, io
from typing import List, Dict
from requests.adapters import HTTPAdapter, Retry
import random
import json
import os
import pandas as pd
from codecarbon import EmissionsTracker
from collections import defaultdict

URL = "http://127.0.0.1:5000/"  ## VERIFICAR QUE ESTO CALCE CON LO ENTREGADO POR LA API!!
TOKEN = "789c16fe57f3bc7f0108525b1ea89a0bb4a4952b" 

# Trial endpoints
ENDPOINT_GET_MESSAGES_TRIAL = URL+"{TASK}/getmessages_trial/{TOKEN}"
ENDPOINT_SUBMIT_DECISIONS_TRIAL = URL+"{SUBTASK}/submit_trial/{TOKEN}/{RUN}"

# Test endpoints
#ENDPOINT_GET_MESSAGES = URL+"{TASK}/getmessages/{TOKEN}"
#ENDPOINT_SUBMIT_DECISIONS = URL+"{SUBTASK}/submit/{TOKEN}/{RUN}"

class Client_taskX:
    def __init__(self, task: str, subtasks: List[str], token: str, number_of_runs: int, tracker: EmissionsTracker):
        # Task in which you participate
        self.task = task
        # Subtasks in which you participate
        self.subtasks = subtasks
        # Token identifier
        self.token = token
        # Number of runs (Max: 3)
        self.number_of_runs = number_of_runs
        self.nicks = []
        # Object to calculate CO2 emissions
        self.tracker = tracker
        self.relevant_cols = ['duration', 'emissions', 'cpu_energy', 'gpu_energy', 'ram_energy', 
            'energy_consumed', 'cpu_count', 'gpu_count', 'cpu_model', 'gpu_model', 'ram_total_size']
        
        self.df_data = []
        self.pipelines = []
        

    # Here a GET request is sent to the server to extract the data.
    def get_messages(self, retries: int, backoff: float) -> Dict:
        session = requests.Session()
        retries = Retry( 
                        total = retries,
                        backoff_factor = backoff,
                        status_forcelist = [500, 502, 503, 504]
                        )
        session.mount('http://', HTTPAdapter(max_retries=retries))
        response = session.get(ENDPOINT_GET_MESSAGES_TRIAL.format(TASK=self.task, TOKEN=self.token))
        if response.status_code != 200:
          print("GET - Status Code " + self.task + ": " + str(response.status_code) + " - Error: " + str(response.text))
          return []
        else:
          #print(response.content)
          return json.loads(response.content)

    
        
    def calculate_decision(self, subtask: int, run=0):
        if self.task == "task1":
            if subtask == 0:
                return self.pipelines[0].predict(self.df_data.text)
            elif subtask == 1:
                return [elem[1] for elem in self.pipelines[1].predict_proba(self.df_data.text)]
        elif self.task == "task2":
            if subtask == 0:
                return self.pipelines[0].predict(self.df_data.text)
            elif subtask == 1:
                return [elem[1] for elem in self.pipelines[1].predict_proba(self.df_data.text)]
            elif subtask == 2:
                return self.pipelines[2].predict(self.df_data.text)
            elif subtask == 3:
                return self.pipelines[3].predict(self.df_data.text)
    
    
    # The POST requests are sent to the server to send predictions and carbon emission data
    def submit_decision(self, subtask: int, messages: List[Dict], emissions:Dict, retries, backoff):
        for run in range(0,self.number_of_runs):
            predictions = self.calculate_decision(subtask, run=run)

            # You must create the appropriate structure to send the predictions according to each subtask
            decisions = pd.Series(predictions,index=self.df_data.nick).to_dict()
            
            #print("las decisiones son:", decisions)


            data = {
                "predictions": decisions,
                "emissions": emissions
            }

            # Session to POST request
            session = requests.Session()
            retries = Retry(
                            total = retries,
                            backoff_factor = backoff,
                            status_forcelist = [500, 502, 503, 504]
                            )
            session.mount('http://', HTTPAdapter(max_retries=retries))


            # For each run, new decisions
            response = session.post(ENDPOINT_SUBMIT_DECISIONS_TRIAL.format(SUBTASK=self.subtasks[subtask], TOKEN=self.token, RUN=run), json=[data])
            if response.status_code != 200:
                print("POST - Status Code " + self.task + ": " + str(response.status_code) + " - Error: " + str(response.text))
            else:
                print("Subtask {}: - run {}".format(self.subtasks[subtask], run))
        

    # Main thread
    def run_taskX(self, retries: int, backoff: float):
        # Get messages for taskX
        messages = self.get_messages(retries, backoff)
        if len(messages) > 0: # cargamos los msges iniciales y lematizamos
            self.df_data = pd.DataFrame.from_dict(messages).drop(columns=["date", "id_message", "round"]).rename(columns={"message": "text"})
            self.df_data = lemmatize_df(self.df_data)
            self.nicks = [message["nick"] for message in messages]
        # If there are no messages
        if len(messages) == 0:
            print("All rounds processed")
            return
        
        while len(messages) > 0:
            print("------------------- Processing round {}".format(messages[0]["round"]))
            # Save subjects
            with open('./data/rounds_trial/round{}.json'.format(messages[0]["round"]), 'w+', encoding='utf8') as json_file:
                json.dump(messages, json_file, ensure_ascii=False)

            # Calculate emissions for each prediction
            self.tracker.start()

            
            emissions = self.tracker.stop()
            df = pd.read_csv("emissions.csv")
            measurements = df.iloc[-1][self.relevant_cols].to_dict()

            self.submit_decision(0, messages, measurements, retries, backoff) # taskXa
            self.submit_decision(1, messages, measurements, retries, backoff) # taskXb
            #self.submit_decision(2, messages, measurements, retries, backoff) # taskXc
            #self.submit_decision(3, messages, measurements, retries, backoff) # taskXd

            # Only one GET request for each round
            messages = self.get_messages(retries, backoff)
            
            # ahora vamos a concatenar los nuevos msges lematizados al dataframe con los textos
            new_texts = []
            nicks = []
            for msg in messages:
                nicks.append(msg["nick"])
                new_texts.append(msg["message"])
            lemmatized_new_texts = lemmatize_docs(new_texts)
            
            new_data = dict(zip(nicks, lemmatized_new_texts))
            self.df_data["text"] = self.df_data.apply(lambda row: (row["text"] + " " + new_data.setdefault(row["nick"], "")).strip(), axis=1)

    
        print("All rounds processed")

In [4]:

# Emissions Tracker Config
config = {
    "save_to_file": True,
    "log_level": "DEBUG",
    "tracking_mode": "process",
    "output_dir": ".", 
}
tracker = EmissionsTracker(**config)




[codecarbon INFO @ 10:05:45] [setup] RAM Tracking...
[codecarbon INFO @ 10:05:45] [setup] GPU Tracking...
[codecarbon INFO @ 10:05:46] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:05:46] [setup] CPU Tracking...
[codecarbon DEBUG @ 10:05:46] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : Intel Power Gadget executable not found on win32
[codecarbon DEBUG @ 10:05:46] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : Platform not supported by Intel RAPL Interface
[codecarbon WARNING @ 10:05:46] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon DEBUG @ 10:05:48] CPU : We detect a AMD Ryzen 5 4600H with Radeon Graphics with a TDP of 54 W
[codecarbon INFO @ 10:05:48] CPU Model on constant consumption mode: AMD Ryzen 5 4600H with Radeon Graphics
[codecarbon INFO @ 10:05:48] >>> Tracker's metadata:
[codecarbon INFO @ 10:05:48]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 10

# Requisitos para poder usar pickle en el modelo

In [5]:
# Data processing
import pandas as pd
import numpy as np
import nltk

# Modeling
#import tensorflow as tf

#Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

#Results
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#Features
from sklearn.feature_extraction.text import TfidfVectorizer

#CLF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
# from sklearn.linear_model import LinearRegression

#time xd
from time import time

# text processing
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import re
import string
from nltk.corpus import stopwords

#graficar
import matplotlib.pyplot as plt

import locale
locale.getpreferredencoding = lambda: "UTF-8"
#stopwords and lemmatizer
nltk.download('punkt')
nltk.download('stopwords')
import spacy

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Benja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Benja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# stopwords (sacando mia)
result = set(stopwords.words('spanish'))
print("List of stopwords in spanish:")
print (len(result))
print("\nOmit - 'mía':")
stop_words = set(stopwords.words('spanish')) - set(['mía'])
print("\nList of fresh stopwords in spanish:")
print (len(stop_words))

# lemmatizer
nlp = spacy.load("es_core_news_sm")

List of stopwords in spanish:
313

Omit - 'mía':

List of fresh stopwords in spanish:
312


In [7]:
def lemmatize_df(dF):
    '''
    Lemmatize column 'text' of a dataframe
    '''
    nlp = spacy.load("es_core_news_sm")
    lemma_text_list = []
    for doc in nlp.pipe(dF["text"], disable=["parser", "ner", "ents"]):
        lemma_text_list.append(" ".join(token.lemma_ for token in doc))

    dF.loc[dF.index,"text"] = lemma_text_list
    print("El df lematizado es: ", dF)
    return dF

In [8]:
def lemmatize_docs(docs):
    nlp = spacy.load("es_core_news_sm")
    lemma_text_list = []
    for doc in nlp.pipe(docs, disable=["parser", "ner", "ents"]):
        lemma_text_list.append(" ".join(token.lemma_ for token in doc))
    return lemma_text_list


In [9]:
def lemmatize_list(list_to_lem):
  '''
  Lemmatize column 'text' of a dataframe
  '''
  nlp = spacy.load("es_core_news_sm")
  lemma_text_list = []
  for doc in nlp.pipe(list_to_lem):
      lemma_text_list.append(" ".join(token.lemma_ for token in doc))

  # dF.loc[dF.index,"text"] = lemma_text_list
  return pd.DataFrame(lemma_text_list, columns = ['text'])

# SOLO SE NECESITAN CAMBIAR LAS VARIABLES DE LAS CELDAS POSTERIORES A ESTA

In [10]:
# cargamos el modelo y le quitamos el lematizador
pickled_model = pickle.load(open('multiNB_2a.pkl', 'rb'))
new_model = Pipeline(pickled_model.steps[1:3])
new_model.steps.insert(0,['filler', FunctionTransformer(lambda x: pd.DataFrame(x, columns = ['text']))])

## Recordar correr el servidor local apropiado para la task a realizar

In [11]:
# definimos las tasks y los pipelines a usar, recuerda correr el servidor que corresponde a la task a realizar antes de seguir
TASK = "task2"
SUBTASKS = ["task2a", "task2b"]
PIPELINES_PER_SUBTASK = [new_model, new_model]

In [12]:
## corremos la task
number_runs = 1 # Max: 3 en el servidor real, NO cambiar aquí ya que no hay lógica para entregar diferentes resultados según el run
client_task = Client_taskX(TASK, SUBTASKS, TOKEN, number_runs, tracker)
client_task.pipelines = PIPELINES_PER_SUBTASK
client_task.run_taskX(3, 0.1)

[codecarbon INFO @ 10:05:53] Energy consumed for RAM : 0.000000 kWh. RAM Power : 0.12357044219970705 W
[codecarbon DEBUG @ 10:05:53] RAM : 0.12 W during 0.00 s [measurement time: 0.0070]


El df lematizado es:                                                  text        nick
0  bien ... técnicamente deber ir yo a dormir car...    subject1
1             umm pues como tú explico ... mal : ´ (   subject10
2  Hola , estar realmente mal , no él que hacer c...  subject100
3                        ¿ Cuanto tiempo durar así ?  subject102
4     haber ser día confuso , de momento bien y tú ?  subject103
5            pues el 4 y 20 de el tarde por supuesto  subject105
6  quiza ser mejor que habl con él y él cuentser ...  subject107
7  Holaa a todo cara feliz con ojo sonrient ver q...  subject120
8  uno amigo mío padecer de adicción a ese y él s...  subject206
9                               con depre y ansiedad  subject210
------------------- Processing round 0


[codecarbon INFO @ 10:05:53] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 1.0 W
[codecarbon DEBUG @ 10:05:53] GPU : 3.28 W during 0.02 s [measurement time: 0.3985]
[codecarbon INFO @ 10:05:53] Energy consumed for all CPUs : 0.000003 kWh. All CPUs Power : 27.0 W
[codecarbon DEBUG @ 10:05:53] CPU : 27.00 W during 0.42 s [measurement time: 0.0000]
[codecarbon INFO @ 10:05:53] 0.000003 kWh of electricity used since the begining.
[codecarbon DEBUG @ 10:05:53] last_duration=0.42054319381713867
------------------------
[codecarbon DEBUG @ 10:05:53] We apply an energy mix of 363 g.CO2eq/kWh for Chile
[codecarbon DEBUG @ 10:05:53] EmissionsData(timestamp='2023-04-25T10:05:53', project_name='codecarbon', run_id='cbcdde9b-da5f-4b0a-8c6b-7636ca066156', duration=0.4225451946258545, emissions=1.1467599471177633e-06, emissions_rate=0.002713934418620403, cpu_power=27.0, gpu_power=1.0, ram_power=0.12357044219970705, cpu_energy=3.15407395362854e-06, gpu_energy=4.451142417060004e-09, ram

las decisiones son: {'subject1': 0, 'subject10': 0, 'subject100': 1, 'subject102': 1, 'subject103': 0, 'subject105': 1, 'subject107': 1, 'subject120': 0, 'subject206': 0, 'subject210': 1}
Subtask task2a: - run 0
las decisiones son: {'subject1': 0.19504637982440148, 'subject10': 0.4705461739610953, 'subject100': 0.7829069183938828, 'subject102': 0.623873966188895, 'subject103': 0.2292773111882453, 'subject105': 0.6117726528248979, 'subject107': 0.6585563122127713, 'subject120': 0.31840805084519097, 'subject206': 0.34281849531028097, 'subject210': 0.8629230429868868}
Subtask task2b: - run 0


[codecarbon WARNING @ 10:05:54] Already started tracking
[codecarbon INFO @ 10:05:54] Energy consumed for RAM : 0.000000 kWh. RAM Power : 0.12622976303100586 W
[codecarbon DEBUG @ 10:05:54] RAM : 0.13 W during 0.65 s [measurement time: 0.0080]
[codecarbon INFO @ 10:05:54] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 1.006 W
[codecarbon DEBUG @ 10:05:54] GPU : 1.01 W during 0.67 s [measurement time: 0.0070]
[codecarbon INFO @ 10:05:54] Energy consumed for all CPUs : 0.000008 kWh. All CPUs Power : 27.0 W
[codecarbon DEBUG @ 10:05:54] CPU : 27.00 W during 0.68 s [measurement time: 0.0010]
[codecarbon INFO @ 10:05:54] 0.000008 kWh of electricity used since the begining.
[codecarbon DEBUG @ 10:05:54] last_duration=0.6801576614379883
------------------------
[codecarbon DEBUG @ 10:05:54] We apply an energy mix of 363 g.CO2eq/kWh for Chile
[codecarbon DEBUG @ 10:05:54] EmissionsData(timestamp='2023-04-25T10:05:54', project_name='codecarbon', run_id='cbcdde9b-da5f-4b0a-8c6b-76

...

------------------- Processing round 97
las decisiones son: {'subject1': 0, 'subject10': 1, 'subject100': 1, 'subject102': 1, 'subject103': 0, 'subject105': 1, 'subject107': 1, 'subject120': 1, 'subject206': 1, 'subject210': 1}
Subtask task2a: - run 0
las decisiones son: {'subject1': 0.13049750143124303, 'subject10': 0.9835408608682419, 'subject100': 0.8738327565421639, 'subject102': 0.9697900902347381, 'subject103': 0.39612967121240805, 'subject105': 0.6740442259002608, 'subject107': 0.6467264239817494, 'subject120': 0.508716474379422, 'subject206': 0.7831320820325, 'subject210': 0.8969054627560974}
Subtask task2b: - run 0


[codecarbon WARNING @ 10:06:54] Already started tracking
[codecarbon INFO @ 10:06:54] Energy consumed for RAM : 0.000003 kWh. RAM Power : 0.20755720138549805 W
[codecarbon DEBUG @ 10:06:54] RAM : 0.21 W during 0.58 s [measurement time: 0.0070]
[codecarbon INFO @ 10:06:54] Energy consumed for all GPUs : 0.000017 kWh. All GPUs Power : 1.0040000000000002 W
[codecarbon DEBUG @ 10:06:54] GPU : 1.00 W during 0.59 s [measurement time: 0.0070]
[codecarbon INFO @ 10:06:54] Energy consumed for all CPUs : 0.000462 kWh. All CPUs Power : 27.0 W
[codecarbon DEBUG @ 10:06:54] CPU : 27.00 W during 0.60 s [measurement time: 0.0010]
[codecarbon INFO @ 10:06:54] 0.000482 kWh of electricity used since the begining.
[codecarbon DEBUG @ 10:06:54] last_duration=0.6033024787902832
------------------------
[codecarbon DEBUG @ 10:06:54] We apply an energy mix of 363 g.CO2eq/kWh for Chile
[codecarbon DEBUG @ 10:06:54] EmissionsData(timestamp='2023-04-25T10:06:54', project_name='codecarbon', run_id='cbcdde9b-da5f

------------------- Processing round 98
las decisiones son: {'subject1': 0, 'subject10': 1, 'subject100': 1, 'subject102': 1, 'subject103': 0, 'subject105': 1, 'subject107': 1, 'subject120': 1, 'subject206': 1, 'subject210': 1}
Subtask task2a: - run 0
las decisiones son: {'subject1': 0.13049750143124303, 'subject10': 0.9835408608682419, 'subject100': 0.8738327565421639, 'subject102': 0.9726582096578827, 'subject103': 0.39132578489817116, 'subject105': 0.6741326257951119, 'subject107': 0.6669896905658136, 'subject120': 0.508716474379422, 'subject206': 0.8350004670076884, 'subject210': 0.8969054627560974}
Subtask task2b: - run 0


[codecarbon WARNING @ 10:06:55] Already started tracking
[codecarbon INFO @ 10:06:55] Energy consumed for RAM : 0.000003 kWh. RAM Power : 0.20765447616577148 W
[codecarbon DEBUG @ 10:06:55] RAM : 0.21 W during 0.60 s [measurement time: 0.0070]
[codecarbon INFO @ 10:06:55] Energy consumed for all GPUs : 0.000017 kWh. All GPUs Power : 1.0070000000000001 W
[codecarbon DEBUG @ 10:06:55] GPU : 1.01 W during 0.61 s [measurement time: 0.0070]
[codecarbon INFO @ 10:06:55] Energy consumed for all CPUs : 0.000467 kWh. All CPUs Power : 27.0 W
[codecarbon DEBUG @ 10:06:55] CPU : 27.00 W during 0.63 s [measurement time: 0.0000]
[codecarbon INFO @ 10:06:55] 0.000487 kWh of electricity used since the begining.
[codecarbon DEBUG @ 10:06:55] last_duration=0.6287996768951416
------------------------
[codecarbon DEBUG @ 10:06:55] We apply an energy mix of 363 g.CO2eq/kWh for Chile
[codecarbon DEBUG @ 10:06:55] EmissionsData(timestamp='2023-04-25T10:06:55', project_name='codecarbon', run_id='cbcdde9b-da5f

------------------- Processing round 99
las decisiones son: {'subject1': 0, 'subject10': 1, 'subject100': 1, 'subject102': 1, 'subject103': 0, 'subject105': 1, 'subject107': 1, 'subject120': 1, 'subject206': 1, 'subject210': 1}
Subtask task2a: - run 0
las decisiones son: {'subject1': 0.13049750143124303, 'subject10': 0.9835408608682419, 'subject100': 0.8738327565421639, 'subject102': 0.9748369475984364, 'subject103': 0.39814896304319314, 'subject105': 0.6697009943514338, 'subject107': 0.682676791459777, 'subject120': 0.508716474379422, 'subject206': 0.8477330464768575, 'subject210': 0.8969054627560974}
Subtask task2b: - run 0
All rounds processed
